Working out details mentioned in: https://aosabook.org/en/500L/a-python-interpreter-written-in-python.html

To get immediate feedback, let's build a small interpreter that adds together numbers. Like `7 + 5` and so on.

The interpreter we are going to build is a Stack based interpreter. Meaning we manipulate the stack to execute instructions of our program.

Let's take a look at the instructions we need to add numbers.
1. `LOAD_VALUE`
2. `ADD_TWO_VALUES`
3. `PRINT_ANSWER`

Let's say we are trying to `7 + 5`. We represent this code in instructions like:
```python
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                     ("LOAD_VALUE", 1),
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)
                    ],
    "numbers": [7, 5]
}
```

So in the stack machine, the state would look something like:
1. Empty stack. `[]`
2. `LOAD_VALUE 7`, `[7]`
3. `LOAD_VALUE 5`, `[7, 5]`
4. `ADD_TWO_VALUES`, `[12]` => `7 + 5` = 12
5. `PRINT_ANSWER`, `[]` => pop the result from the stack


Let's build an interpreter for this.

In [1]:
# An Interpreter to add numbers

class Interpreter:
    def __init__(self):
        self.stack = []

    def LOAD_VALUE(self, number: int):
        self.stack.append(number)

    def PRINT_ANSWER(self):
        result = self.stack.pop()
        print(result)

    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)

    def run_code(self, what_to_execute: object):
        instructions = what_to_execute["instructions"]
        numbers = what_to_execute["numbers"]

        for each_step in instructions:
            instruction, argument = each_step
            if instruction == "LOAD_VALUE":
                self.LOAD_VALUE(numbers[argument])
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()

what_to_execute = {
    "instructions": [("LOAD_VALUE", 0), ("LOAD_VALUE", 1), ("ADD_TWO_VALUES", None), ("PRINT_ANSWER", None)],
    "numbers": [7, 5]
}

interpreter = Interpreter()
interpreter.run_code(what_to_execute) # should print 12

12


Using the same interpreter what if we want to add *n* numbers? We can modify the instructions such that it can add numbers. Let's look at that.

In [4]:
what_to_execute = {
    "instructions": [
        ("LOAD_VALUE", 0), ("LOAD_VALUE", 1), ("ADD_TWO_VALUES", None), ("LOAD_VALUE", 2), ("LOAD_VALUE", 3),
        ("ADD_TWO_VALUES", None), ("ADD_TWO_VALUES", None), ("PRINT_ANSWER", None)
    ],
    "numbers": [100, 200, 1500, 40]
}

interpreter = Interpreter()
interpreter.run_code(what_to_execute) # should print 1840

1840


### Variables

Next lets add Variables to our interpreter. Variables require an instruction for storing the value of a variable, `STORE_NAME`; an instruction for retrieving it `LOAD_NAME`; and a mapping from variable names to values.

For now, we'll ignore the namespaces and scoping, so that we can store the variable mapping on the interpreter object itself.

Finally we have to make sure that `what_to_execute` has a list of variable names, in addition to its list of constants.

### Example
```python
def s():
    a = 1
    b = 2
    print(a + b)

# s is transformed into:

what_to_execute = {
    "instructions": [
        ("LOAD_VALUE", 0),
        ("STORE_NAME", 0),
        ("LOAD_VALUE", 1),
        ("STORE_NAME", 1),
        ("LOAD_NAME", 0),
        ("LOAD_NAME", 1),
        ("ADD_TWO_VALUES", None),
        ("PRINT_ANSWER", None)
    ],
    "numbers": [1, 2],
    "names": ["a", "b"]
}
```

To keep track of what names are bound to what values, we'll add an `environment` dictionary to the `__init__` method. We'll also add `STORE_NAME` and `LOAD_NAME`. These methods first look up variables in question and then use the dictionary to store or retrieve its value.

**NOTE:** The arguments of the instructions can now mean two different things. They can either be an index into the "numbers" list, or they can be an index into the "names" list. The interpreter knows which it should be by checking what instruction it's executing. We'll break out this logic -- and the mapping of instruction to what their arguments mean -- into a separate method.

In [9]:
# Updated code for Interpreter with support for Variable instructions
class Interpreter:
    def __init__(self):
        self.stack = []
        self.environment = {}

    def LOAD_VALUE(self, number: int):
        self.stack.append(number)

    def PRINT_ANSWER(self):
        result = self.stack.pop()
        print(result)

    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)

    def STORE_NAME(self, name):
        val = self.stack.pop()
        self.environment[name] = val

    def LOAD_NAME(self, name):
        val = self.environment[name]
        self.stack.append(val)

    def parse_argument(self, instruction, argument, what_to_execute):
        # Understand what the argument to each instruction means
        numbers = ["LOAD_VALUE"]
        names = ["LOAD_NAME", "STORE_NAME"]

        if instruction in numbers:
            argument = what_to_execute["numbers"][argument]
        elif instruction in names:
            argument = what_to_execute["names"][argument]
        
        return argument

    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)

            if instruction == "LOAD_VALUE":
                self.LOAD_VALUE(argument)
            elif instruction == "LOAD_NAME":
                self.LOAD_NAME(argument)
            elif instruction == "STORE_NAME":
                self.STORE_NAME(argument)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()

    def execute_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)
            bytecode_method = getattr(self, instruction)
            if argument is None:
                bytecode_method()
            else:
                bytecode_method(argument)


what_to_execute = {
    "instructions": [
        ("LOAD_VALUE", 0),
        ("STORE_NAME", 0),
        ("LOAD_VALUE", 1),
        ("STORE_NAME", 1),
        ("LOAD_NAME", 0),
        ("LOAD_NAME", 1),
        ("ADD_TWO_VALUES", None),
        ("PRINT_ANSWER", None)
    ],
    "numbers": [1, 2],
    "names": ["a", "b"]
}

interpreter = Interpreter()
interpreter.run_code(what_to_execute)
# interpreter.execute_code(what_to_execute)


3


Even with just five instructions, the `run_code` method is starting to get tedious. If we have to add one more instruction, then we have to add an if branch and that makes our code bloated. To fix this, we can use Python's dynamic method lookup. We can use the `getattr` function to look up the method on the fly. Let's implement it in the above cell. Check out the `execute_code` method.